Sua  tarefa  será  gerar  a  matriz  termo-documento  usando  TF-IDF  por  meio  da  aplicação  das 
fórmulas TF-IDF na matriz termo-documento criada com a utilização do algoritmo Bag of Words. Sobre 
o Corpus que recuperamos anteriormente. 
 

In [ ]:
from os import X_OK
#Aluno: Fernando Purkott
from bs4 import BeautifulSoup
import requests
import spacy
from spacy.language import Language
import re
import string
import nltk
from nltk.corpus import stopwords
import numpy as np



#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



@Language.component("Space")
def set_custom_boundaries(doc):
     for token in doc[:-1]:
         if token.text.__contains__('  '):
             doc[token.i+1].is_sent_start = True
     return doc
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('Space', before="parser")

def excluiNovaLinha(value):
    return ''.join(value.splitlines())



blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'style'
]

def scrape(URL):
  blacklist = [
    '[document]',
    'noscript',
    'header',
    'html',
    'meta',
    'head', 
    'input',
    'script',
    'style'
  ]

  req = requests.get(URL)
  soup = BeautifulSoup(req.content)


  txt = soup.find_all(text=True)
  clean_txt = ""

  for t in txt:

    
    if t.parent.name not in blacklist:
      t = t.replace('—', ' ')
      t = t.replace(',', ' ')
      t = t.replace('.', ' ')
      t = t.replace("'", '')
      t = t.replace("’", ' ')
      t = t.replace("‘", ' ')
      t = t.replace('-', ' ')
      clean_txt += '{} '.format(t)

  txt_doc = nlp(excluiNovaLinha(clean_txt))


  sentences = list(txt_doc.sents)
  return sentences

def criaVocab(sentences):
  vocab = []
  sentenceWords = []
  for s in sentences:
    for i in s:
      sentenceWords = str(i).split()
      for j in sentenceWords:
        j = j.lower()
        if j not in stop_words and j.translate(str.maketrans('', '', string.punctuation)) != "":
          vocab.append(j.translate(str.maketrans('', '', string.punctuation)))  
      sentenceWords = []
    vocab = unico(vocab)
    return vocab
  

def unico(words):
  seen = set()
  return [x for x in words if not (x in seen or seen.add(x))]

def vectorize(tokens, vocab):
    vector=[]
    for w in vocab:
      x = 0
      for i in tokens:
        if i == w:
          x +=1
      vector.append(x)
    return vector

def BagOfWords(sentences):
  vocab = criaVocab(sentences)
  numSentences = 1
  for i in sentences:
    for j in i:
      numSentences = numSentences + 1
  bagofwords = [None] * numSentences
  bagofwords[0] = vocab
  currentSentence = 1
  for i in sentences:
    for j in i:
      tokens = str(j).split()
      for z in range(len(tokens)):
        tokens[z] = tokens[z].lower()  
      bagofwords[currentSentence] = vectorize(tokens, vocab)
      currentSentence = currentSentence + 1
  return bagofwords
      
def TF(sentences, bagofwords): 
  tf = np.zeros((len(bagofwords),len(bagofwords[0])))
  currentSentence = 0
  for i in sentences:
    for j in i:
      numberofwords = len(str(j).split())
      for word in str(j).split():
        if word in bagofwords[0]:
          tf[currentSentence][bagofwords[0].index(word.lower())] = bagofwords[0].index(word.lower()) / numberofwords
      currentSentence = 1
  return tf

def IDF(bagofwords):
  errors = []
  idf = np.zeros(len(bagofwords[0]))
  currentSentence = 0
  for w in bagofwords[0]:
    x = 0      
    for i in bagofwords[1:]:
      if i[bagofwords[0].index(w)] > 0: x += 1
    try:     
      idf[bagofwords[0].index(w)] =  np.log10(len(bagofwords[1:]) / x)
    except:
       idf[bagofwords[0].index(w)] = 1
  return idf

def TFIDF(tf,idf):
  tfidf = tf.copy()
  for i in range(len(tfidf)):
    for j in range(len(tfidf[0])):
      tfidf[i][j] = tfidf[i][j] * idf[j]
  return tfidf
sentences1 = scrape("https://www.ibm.com/cloud/learn/natural-language-processing")
sentences2 = scrape("https://hbr.org/2022/04/the-power-of-natural-language-processing")
sentences3 = scrape("https://link.springer.com/article/10.1007/s11042-022-13428-4")
sentences4 = scrape("https://www.techtarget.com/searchenterpriseai/definition/natural-language-processing-NLP")
sentences5 = scrape("https://monkeylearn.com/natural-language-processing/")

sentences = [sentences1, sentences2, sentences3, sentences4, sentences5]
bagofwords = BagOfWords(sentences)
tf = TF(sentences, bagofwords)
idf = IDF(bagofwords)
tfidf = TFIDF(tf,idf)
for i in tfidf:
  print(i)

  




    
